In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('Motocycles')
n_line = df.shape[0]
df.drop_duplicates(keep = 'first', inplace = True)
df = df.reset_index(drop = True)
print(f"Было удалено {n_line - df.shape[0]} строк-дубликатов")
print(f"shape = {df.shape}")
df.head()

Было удалено 413 строк-дубликатов
shape = (18977, 12)


,Марка,Модель,Год,Город,Пробег(км),Объем двигателя (см³),Мощность двигателя (л.с.),Цилиндров,Коробка,Привод,Цвет,Цена(руб)
0,ABM,ABM Jazz 125,2015,Вязьма,7400,125,NaN,NaN,4 передачи,NaN,чёрный,36000
1,ABM,ABM ZR 200,2013,Егорьевск,15000,200,19.0,NaN,5 передач,цепь,чёрный,63000
2,ABM,ABM ZR 200,2014,Сиверский,1500,200,15.0,NaN,5 передач,цепь,зелёный,90000
3,ABM,ABM Raptor 200,2011,Альметьевск,1000,200,17.0,1 цилиндр,5 передач,цепь,чёрный,90000
4,ABM,ABM Alpha 110,2023,Татарская Каргала,700,110,9.0,1 цилиндр,4 передачи,цепь,чёрный,62000


In [3]:
df.isnull().sum()

Марка                           0
Модель                          0
Год                             0
Город                           0
Пробег(км)                      0
Объем двигателя (см³)           0
Мощность двигателя (л.с.)    3430
Цилиндров                    5405
Коробка                      3475
Привод                       3197
Цвет                            0
Цена(руб)                       0
dtype: int64

Пропуски есть, их количество - от ~15 до ~30% данных по столбцу.

In [4]:
print(f"Уникальных моделей: {df['Модель'].nunique()}")

Уникальных моделей: 1831


Количество уникальных моделей намного ~20000, а значит они повторяются часто, что позволит нам заполнить большинство пропусков.

**Заполнять пропуски буду медианой (модой) по модели**

In [5]:
def sep_cil1(line):
    """Данная функция оставляет значение количества цилиндров, удаляя форму двигателя"""
    if type(line) != float:
        x = line.split('/')
        x[0] = x[0].split()
        return(x[0][0])
    else:
        return(np.nan)

In [6]:
def sep_cil2(line):
    """Данная функция оставляет форму двигателя, если она указана"""
    if type(line) != float:
        x = line.split('/')
        if len(x) == 2:
            return(x[1])
        else:
            return(np.nan)
    else:
        return(np.nan)

In [7]:
df['Форма двигателя'] = df['Цилиндров'].apply(sep_cil2)
df['Цилиндров'] = df['Цилиндров'].apply(sep_cil1).astype('float64')

Как и прописано в использованных функциях, оставим количество цилиндров в столбце 'Цилиндров'

Дополнительную информацию запишем в столбец 'Форма двигателя'

In [8]:
df.groupby('Коробка')[['Марка']].count()

,Марка
Коробка,
1 передача,13
2 передачи,12
2-скоростной автомат,3
3 передачи,33
3-скоростной автомат,6
4 передачи,1249
4 прямых и задняя,62
5 передач,4456
5 прямых и задняя,79


Посмотрим на значения коробки передач. Легко разделить на 4 группы - МКПП, АКПП, Вариатор, Робот. Но заметим, что у МКПП очень варьируются значения количества передач, поэтому при i передач, будем присваивать название МКПП-i

In [9]:
def Box_coder(line):
    """Приводит данные о коробке передач к более общему виду"""
    if type(line) != float:
        if 'автомат' in line or 'АКПП' in line:
            return('АКПП')
        elif line[0].isnumeric():
            return(f"МКПП-{line.split()[0]}")
        elif 'Робот' in line:
            return('Робот')
        else:
            return('Вариатор')
            
    else:
        return(np.nan)

In [10]:
df['Коробка'] = df['Коробка'].apply(Box_coder)

In [11]:
gb1 = df.groupby('Модель')[['Мощность двигателя (л.с.)']].median()
gb2 = df.groupby('Модель')[['Цилиндров', 'Коробка', 'Привод', 'Форма двигателя']].agg(pd.Series.mode)
gb = pd.concat((gb1, gb2), axis = 1)
gb.head()
# Создаем отдельную таблицу со сгруппированными по модели данными
# Для категориальных столбцов применяем моду, для числовых - медиану

,Мощность двигателя (л.с.),Цилиндров,Коробка,Привод,Форма двигателя
Модель,,,,,
ABM Alpha 110,8.0,1.0,МКПП-4,цепь,рядное
ABM Alpha 50,7.0,"[1.0, 2.0]","[МКПП-1, МКПП-4]",цепь,рядное
ABM Cross,7.0,1.0,МКПП-2,кардан,[]
ABM GX 250,21.0,1.0,МКПП-5,цепь,v-образное
ABM Jazz 125,11.0,1.0,"[МКПП-4, МКПП-5]",цепь,рядное


In [12]:
gb.isnull().sum()

Мощность двигателя (л.с.)    94
Цилиндров                     0
Коробка                       0
Привод                        0
Форма двигателя               0
dtype: int64

In [13]:
gb = gb.dropna()

Пропуски есть только в столбцах, которые были заполнены медианным значением по марке =>

Скорее всего там находятся модели, в которых полностью отсутствует информация о мощности двигателя => выбросим эти марки

In [14]:
def NoList1(line):
    if type(line) != np.float64:
        if len(line) == 0:
            return(np.nan)
        else:
            a = [2, 1, 4]
            for item in a:
                if item in list(line):
                    return(item)
                    break
            return(line[0])
    return(line)

In [15]:
def NoList2(line):
    if type(line) != str:
        if len(line) == 0:
            return(np.nan)
        else:
            a = ['МКПП-6', 'МКПП-5', 'МКПП-4']
            for item in a:
                if item in list(line):
                    return(item)
                    break
            return(line[0])
    return(line)

In [16]:
def NoList3(line):
    if type(line) != str:
        if len(line) == 0:
            return(np.nan)
        else:
            a = ['цепь, кардан']
            for item in a:
                if item in list(line):
                    return(item)
                    break
            return(line[0])
    return(line)

In [17]:
def NoList4(line):
    if type(line) != str:
        if len(line) == 0:
            return(np.nan)
        else:
            a = ['рядное', 'v-образное', 'оппозитное']
            for item in a:
                if item in list(line):
                    return(item)
                    break
            return(line[0])
    return(line)

In [18]:
gb['Цилиндров'] = gb['Цилиндров'].apply(NoList1)
gb['Коробка'] = gb['Коробка'].apply(NoList2)
gb['Привод'] = gb['Привод'].apply(NoList3)
gb['Форма двигателя'] = gb['Форма двигателя'].apply(NoList4)

In [19]:
gb.isnull().sum()

Мощность двигателя (л.с.)      0
Цилиндров                     76
Коробка                       66
Привод                        59
Форма двигателя              325
dtype: int64

In [20]:
gb = gb.dropna()

In [21]:
drop_ind = []
models = list(gb.index)
for i in range(len(df)):
    if df.iloc[i]['Модель'] not in models:
        drop_ind.append(i)
df = df.drop(drop_ind)
df = df.reset_index(drop = True)

In [22]:
for i in range(len(df)):
    for col in gb.columns:
        if df[col].loc[i] != df[col].loc[i]:
            df[col].loc[i] = gb[gb.index == df['Модель'].loc[i]][col][0]

In [23]:
df.head()

,Марка,Модель,Год,Город,Пробег(км),Объем двигателя (см³),Мощность двигателя (л.с.),Цилиндров,Коробка,Привод,Цвет,Цена(руб),Форма двигателя
0,ABM,ABM Jazz 125,2015,Вязьма,7400,125,11.0,1.0,МКПП-4,цепь,чёрный,36000,рядное
1,ABM,ABM ZR 200,2013,Егорьевск,15000,200,19.0,1.0,МКПП-5,цепь,чёрный,63000,v-образное
2,ABM,ABM ZR 200,2014,Сиверский,1500,200,15.0,1.0,МКПП-5,цепь,зелёный,90000,v-образное
3,ABM,ABM Alpha 110,2023,Татарская Каргала,700,110,9.0,1.0,МКПП-4,цепь,чёрный,62000,рядное
4,ABM,ABM Alpha 110,2023,Фрязино,500,110,7.0,1.0,МКПП-4,цепь,чёрный,75000,рядное


In [24]:
df.isnull().sum()

Марка                        0
Модель                       0
Год                          0
Город                        0
Пробег(км)                   0
Объем двигателя (см³)        0
Мощность двигателя (л.с.)    0
Цилиндров                    0
Коробка                      0
Привод                       0
Цвет                         0
Цена(руб)                    0
Форма двигателя              0
dtype: int64

In [27]:
df.to_csv('Data', mode = 'w')